In [1]:
from autoencoder3 import autoencoder3
from autoencoder5 import autoencoder5
import torch

import numpy as np
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt

INDEX = 1#THIS MUST BE CHANGED IN THE SMALL FILES!

In [2]:
### Run this cell

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training  data
trainsetMNIST = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
train_MNIST = torch.utils.data.DataLoader(trainsetMNIST, batch_size=64, shuffle=True)

# Download and load the test data
testsetMNIST = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
test_MNIST = torch.utils.data.DataLoader(testsetMNIST, batch_size=64, shuffle=True)

In [3]:

# Download and load the training data
trainsetFMNIST = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
train_FMNIST = torch.utils.data.DataLoader(trainsetFMNIST, batch_size=64, shuffle=True)

# Download and load the test data
testsetFMNIST = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
test_FMNIST = torch.utils.data.DataLoader(testsetFMNIST, batch_size=64, shuffle=True)

In [4]:
import copy


valid_MNIST = copy.deepcopy(train_MNIST)  # Creates a copy of the object

#We take the first 45k images for training
train_MNIST.dataset.data = train_MNIST.dataset.data[:45000,:,:]

#And the rest for validation
valid_MNIST.dataset.data = valid_MNIST.dataset.data[45000:,:,:]


In [5]:
import copy

valid_FMNIST = copy.deepcopy(train_FMNIST)  # Creates a copy of the object

#We take the first 45k images for training
train_FMNIST.dataset.data = train_FMNIST.dataset.data[:45000,:,:]

#And the rest for validation
valid_FMNIST.dataset.data = valid_FMNIST.dataset.data[45000:,:,:]

In [6]:
INDEX = 0 # INDEX = 0 for MNIST and 1 for FMNIST!

train_set = [train_MNIST, train_FMNIST]
valid_set = [valid_MNIST, valid_FMNIST]
test_set = [test_MNIST, test_FMNIST]

In [7]:
def save_models(projected_dimensions, models_autoencoder3, models_autoencoder5, INDEX):
    if INDEX == 0:  # MNIST
        for i in range(len(models_autoencoder3)):
            torch.save(models_autoencoder3[i].state_dict(), f'model_autoencoder3_MNIST_{projected_dimensions[i]}.pth')
            torch.save(models_autoencoder5[i].state_dict(), f'model_autoencoder5_MNIST_{projected_dimensions[i]}.pth')
    elif INDEX == 1:  # FMNIST
        for i in range(len(models_autoencoder3)):
            torch.save(models_autoencoder3[i].state_dict(), f'model_autoencoder3_FMNIST_{projected_dimensions[i]}.pth')
            torch.save(models_autoencoder5[i].state_dict(), f'model_autoencoder5_FMNIST_{projected_dimensions[i]}.pth')
    else:
        raise ValueError("While saving the models, INDEX should be either 0 or 1.")


def load_models(projected_dimensions, autoencoder3, autoencoder5, models_autoencoder3, models_autoencoder5, INDEX):
    if INDEX == 0:  # MNIST
        dataset0 = "MNIST"
    else:
        dataset0 = "FMNIST"

    for i in range(len(projected_dimensions)):
        state_dict_3 = torch.load(f'model_autoencoder3_{dataset0}_{projected_dimensions[i]}.pth')
        model_3 = autoencoder3(projected_dimensions[i], INDEX)  # Assuming autoencoder3 is defined elsewhere
        model_3.load_state_dict(state_dict_3)
        models_autoencoder3.append(model_3)

        state_dict_5 = torch.load(f'model_autoencoder5_{dataset0}_{projected_dimensions[i]}.pth')
        model_5 = autoencoder5(projected_dimensions[i], INDEX)  # Assuming autoencoder5 is defined elsewhere
        model_5.load_state_dict(state_dict_5)
        models_autoencoder5.append(model_5)


def evaluate_models(models, train, valid, test):
    for n_layers in range(len(models)):
        for i, model in enumerate(models[n_layers]):
            n_lay = 3 if n_layers == 0 else 5
            print(f"Autoenc{n_lay}, train-{projected_dimensions[i]}. Projections:", model.eval_performance(train))
            print(f"Autoenc{n_lay}, val-{projected_dimensions[i]}. Projections:", model.eval_performance(valid))
            print(f"Autoenc{n_lay}, test-{projected_dimensions[i]}. Projections:", model.eval_performance(test), "\n")

In [8]:
projected_dimensions = [15, 30, 50, 100] # This are the dimensions we are going to project the data into
models_autoencoder3 = []
models_autoencoder5 = []
load_models(projected_dimensions, autoencoder3, autoencoder5, models_autoencoder3, models_autoencoder5, INDEX)

In [9]:
# models_autoencoder5[0].eval_performance(train_MNIST)

In [10]:
# models_autoencoder3[0].eval_performance(train_MNIST)

In [11]:
models = [models_autoencoder3, models_autoencoder5]
evaluate_models(models, train_set[INDEX], valid_set[INDEX], test_set[INDEX]) # TO DO change test to F and MNIST

Autoenc3, train-15. Projections: tensor(7.7331)
Autoenc3, val-15. Projections: tensor(6.8064)
Autoenc3, test-15. Projections: tensor(7.1767) 

Autoenc3, train-30. Projections: tensor(6.3135)
Autoenc3, val-30. Projections: tensor(6.4261)
Autoenc3, test-30. Projections: tensor(6.0756) 

Autoenc3, train-50. Projections: tensor(8.6932)
Autoenc3, val-50. Projections: tensor(8.0335)
Autoenc3, test-50. Projections: tensor(8.1091) 

Autoenc3, train-100. Projections: tensor(7.2512)
Autoenc3, val-100. Projections: tensor(8.3514)
Autoenc3, test-100. Projections: tensor(8.1916) 

Autoenc5, train-15. Projections: tensor(6.8444)
Autoenc5, val-15. Projections: tensor(6.0797)
Autoenc5, test-15. Projections: tensor(6.5974) 

Autoenc5, train-30. Projections: tensor(7.6245)
Autoenc5, val-30. Projections: tensor(6.4958)
Autoenc5, test-30. Projections: tensor(7.0109) 

Autoenc5, train-50. Projections: tensor(6.5752)
Autoenc5, val-50. Projections: tensor(6.7558)
Autoenc5, test-50. Projections: tensor(7.3093